In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [2]:
df_pedidos = pd.read_parquet("../../../data/raw/tb_pedidos.parquet")
df_pedidos["CD_OP"] = df_pedidos["CD_PEDIDO"] + "/" + df_pedidos["CD_ITEM"]

df_merge_base = pd.read_parquet("df_final_base.parquet")
list_itens = list(df_merge_base.CD_OP.unique())


In [3]:
df_pedidos.columns

Index(['CD_PEDIDO', 'CD_ITEM', 'ID_CLIENTE', 'CD_PALETE', 'CD_TIPOFT2',
       'CD_REFERENCIA', 'DT_ENTREGA2', 'DT_ENTREGAORIGINAL', 'FL_AMARRADO',
       'FL_CHAPA', 'FL_EXIGELAUDO', 'FL_PALETIZADO', 'FL_REFILADO',
       'FL_RESINAINTERNA', 'FL_SUSPENSO', 'FL_TIPOENTREGA', 'FL_SUSPOUCANCEL',
       'ST_PEDIDO', 'TX_DESCRSTATUSPEDIDO', 'TX_DESCRTIPODOPEDIDO',
       'TX_DESCTIPOENTREGA', 'TX_TIPOABNT', 'TX_COMPOSICAO', 'CD_ESPELHO',
       'CD_FILME', 'CD_FACA', 'QT_ARRANJO', 'QT_COBBINTMAXIMO', 'QT_NRCORES',
       'QT_PEDIDA', 'QT_PEDIDAMAX', 'QT_PEDIDAMIN', 'QT_PROLONGLAP',
       'QT_PECASPORPACOTE', 'QT_PECASPORPALETE', 'QT_UNIDADESPORPALETE',
       'QT_PACOTESPORPALETE', 'VL_LAPINTERNO', 'VL_LAPNOCOMP', 'VL_GRAMATURA',
       'VL_COMPRIMENTO', 'VL_LARGURA', 'VL_ALTURAINTERNA', 'VL_REFUGOCLIENTE',
       'VL_PESOPECA', 'VL_PESOCAIXA', 'VL_AREABRUTAPECA', 'VL_AREALIQUIDAPECA',
       'VL_AREABRUTACHAPA', 'VL_AREALIQUIDACHAPA', 'VL_VOLUMEFECHADOPEDIDO',
       'VL_VOLUMEPACOTEFECH

In [4]:
df_pedidos = df_pedidos[df_pedidos["DT_ENTREGAORIGINAL"]>= "2024-01-01"]

In [5]:
df_pedidos = df_pedidos[df_pedidos["FL_SUSPOUCANCEL"] == "0"]

In [6]:
df_pedidos= df_pedidos[df_pedidos["ST_PEDIDO"] != "4"]

In [7]:
df_pedidos= df_pedidos[df_pedidos["FL_SUSPENSO"] == "0"]

In [8]:
df_pedidos.shape

(193490, 88)

In [9]:
df_pedidos["PERC_VAR_PEDIDA"] = (
    (df_pedidos["QT_PEDIDAMAX"] - df_pedidos["QT_PEDIDA"])
    / df_pedidos["QT_PEDIDA"]
    * 100
)

In [10]:
df_pedidos.VL_GRAMATURA.value_counts()

VL_GRAMATURA
416.0    26608
446.0    22627
638.0    14194
367.0    13571
369.0     8890
         ...  
858.0        1
642.0        1
462.0        1
489.0        1
640.0        1
Name: count, Length: 182, dtype: int64

- VL_CONSUMOCORES_TOTAL ->DROP QT_CONSUMOCOR1	QT_CONSUMOCOR2	QT_CONSUMOCOR3	QT_CONSUMOCOR4
- QT_PROLONGLAP - > Transformar em flag, >0 fl =1, =0 fl =0
- VL_LAPNOCOMP -> REPlace =1, 1 e =<0, =0
- FL_LAP_INTERNO -> REPlace =1, 1 e =<0, =0

In [11]:
df_pedidos = df_pedidos[df_pedidos["FL_CHAPA"]!="1"]

In [12]:
df_pedidos.FL_CHAPA.value_counts()

FL_CHAPA
0    150596
Name: count, dtype: Int64

In [13]:
cols_drop = [
    "ID_CLIENTE",
    "FL_CHAPA",
    "VL_ALTURAPACOTE",
    "VL_ALTURAPALETEFECHADO",
    "VL_COMPPALETEFECHADO",
    "VL_COMPPACOTE",
    "VL_LARGPACOTE",
    "VL_LARGPALETEFECHADO",
    "VL_PACOTESCOMPRIMENTO",
    "VL_PACOTESLARGURA",
    "VL_PACOTESALTURA",
    "TX_DESCRTIPODOPEDIDO",
    "TX_DESCRSTATUSPEDIDO",
    "TX_DESCRSTATUSPEDIDO",
    "TX_DESCTIPOENTREGA",
    "DT_ENTREGA2",
    "DT_ENTREGAORIGINAL",
    "CD_REFERENCIA",
    "CD_PALETE",
    "FL_PALETIZADO",
    "FL_TIPOENTREGA",
    "FL_SUSPOUCANCEL",
    "ST_PEDIDO",
    "FL_SUSPENSO",
    "QT_PECASPORPACOTE",
    "QT_PECASPORPALETE",
    "QT_UNIDADESPORPALETE",
    "QT_PACOTESPORPALETE",
    "VL_VOLUMEPACOTEFECHADOM3",
    "VL_VOLUMEPALETEFECHADOM3",
    "VL_VOLUMEFECHADOPEDIDO",
    "QT_PEDIDAMAX",
    "QT_PEDIDAMIN",
]


In [14]:
df_pedidos_fil =df_pedidos[sorted(df_pedidos.columns)].drop(columns=cols_drop)


In [15]:
df_pedidos[["VL_GRAMATURA", "VL_COMPRESSAO", "QT_COBBINTMAXIMO", "VL_COLUNAMINIMO"]].describe()

,VL_GRAMATURA,VL_COMPRESSAO,QT_COBBINTMAXIMO,VL_COLUNAMINIMO
count,150596.000000,150596.000000,150596.000000,150596.000000
mean,470.238499,47.631929,30.359658,6.159932
std,151.792843,133.647386,14.290699,2.081767
min,0.000000,0.000000,0.000000,0.000000
25%,416.000000,0.000000,18.000000,5.000000
50%,446.000000,0.000000,30.000000,6.000000
75%,524.000000,0.000000,45.000000,7.000000
max,1012.000000,936.000000,60.000000,14.000000


In [16]:
# Renomear colunas para melhor semântica
df_pedidos_fil = df_pedidos_fil.rename(
    columns={
        "FL_EXIGELAUDO": "FL_TESTE_EXIGELAUDO",
        "VL_COLUNAMINIMO": "VL_COLUNAMINIMO",
        "QT_COBBINTMAXIMO": "VL_COBBINTMAXIMO",
        "VL_COMPRESSAO": "VL_COMPRESSAO",
        "VL_GRAMATURA": "VL_GRAMATURA",
        "CD_ESPELHO": "CAT_ESPELHO",
        "CD_FILME": "CAT_FILME",
        "CD_TIPOFT2": "FL_CONTROLE_ESPECIAL_IMPRESSAO",
        "VL_LAPINTERNO": "FL_LAP_INTERNO",
        "VL_LAPNOCOMP": "FL_LAP_NO_COMPR",
        "TX_COMPOSICAO": "CAT_COMPOSICAO",
    }
)


In [17]:
# 1. Converter LAP flags: -1 → 0 (padronizar como binário)
colunas_lap = ["FL_LAP_INTERNO", "FL_LAP_NO_COMPR"]
for col in colunas_lap:
    if col in df_pedidos_fil.columns:
        df_pedidos_fil[col] = df_pedidos_fil[col].replace(-1, 0)
        print(f"{col}: valores únicos = {sorted(df_pedidos_fil[col].unique())}")

# 2. Converter FL_PROLONG_LAP para binário
if "FL_PROLONG_LAP" in df_pedidos_fil.columns:
    df_pedidos_fil["FL_PROLONG_LAP"] = (df_pedidos_fil["FL_PROLONG_LAP"] > 0).astype(
        int
    )
    print(
        f"FL_PROLONG_LAP: {df_pedidos_fil['FL_PROLONG_LAP'].value_counts().to_dict()}"
    )

# 3. Padronizar FL_CONTROLE_ESPECIAL_IMPRESSAO: -1/2 → 0/1
if "FL_CONTROLE_ESPECIAL_IMPRESSAO" in df_pedidos_fil.columns:
    df_pedidos_fil["FL_CONTROLE_ESPECIAL_IMPRESSAO"] = (
        df_pedidos_fil["FL_CONTROLE_ESPECIAL_IMPRESSAO"]
        .astype(str)
        .replace({"-1": "0", "2": "1"})
    )
    print(
        f"FL_CONTROLE_ESPECIAL_IMPRESSAO: {df_pedidos_fil['FL_CONTROLE_ESPECIAL_IMPRESSAO'].value_counts().to_dict()}"
    )

# 4. Converter tipos numéricos
cols_int = [
    "FL_LAP_INTERNO",
    "FL_LAP_NO_COMPR",
    "FL_PROLONG_LAP",
    "QT_ARRANJO",
    "QT_NRCORES",
]
for col in cols_int:
    if col in df_pedidos_fil.columns:
        df_pedidos_fil[col] = (
            pd.to_numeric(df_pedidos_fil[col]).fillna(0).astype(int)
        )



FL_LAP_INTERNO: valores únicos = [np.float64(0.0), np.float64(1.0)]
FL_LAP_NO_COMPR: valores únicos = [np.float64(0.0), np.float64(1.0)]
FL_CONTROLE_ESPECIAL_IMPRESSAO: {'0': 150271, '1': 325}


In [18]:
df_pedidos_fil["VL_CONSUMO_COR_TOTAL"] = df_pedidos_fil["QT_CONSUMOCOR1"]+df_pedidos_fil["QT_CONSUMOCOR2"]+df_pedidos_fil["QT_CONSUMOCOR3"]+df_pedidos_fil["QT_CONSUMOCOR4"]

In [19]:
df_pedidos_fil.QT_PROLONGLAP.value_counts()

QT_PROLONGLAP
0.0     136370
50.0     14216
30.0        10
Name: count, dtype: int64

In [20]:
df_pedidos_fil["FL_PROLONGLAP"] = (
    df_pedidos_fil["QT_PROLONGLAP"].replace({50: 1, 30: 1}).sort_values()
)


In [21]:
cols_drop_color = [
    "QT_CONSUMOCOR1",
    "QT_CONSUMOCOR2",
    "QT_CONSUMOCOR3",
    "QT_CONSUMOCOR4",
    "QT_PROLONGLAP",
]

df_pedidos_fil.drop(columns=cols_drop_color,inplace=True)

In [22]:
df_pedidos_fil

,CAT_ESPELHO,CD_FACA,CAT_FILME,CD_ITEM,CD_OP,CD_PEDIDO,FL_CONTROLE_ESPECIAL_IMPRESSAO,FL_AMARRADO,FL_TESTE_EXIGELAUDO,FL_REFILADO,FL_RESINAINTERNA,PERC_VAR_PEDIDA,QT_ARRANJO,VL_COBBINTMAXIMO,QT_NRCORES,QT_PEDIDA,CAT_COMPOSICAO,TX_TIPOABNT,VL_ALTURAINTERNA,VL_ALTURAINTERNA_REAL,VL_AREABRUTACHAPA,VL_AREABRUTAPECA,VL_AREABRUTAPECACOMREFILOS,VL_AREALIQUIDACHAPA,VL_AREALIQUIDAPECA,VL_COLUNAMINIMO,VL_COMPPECA,VL_COMPRESSAO,VL_COMPRIMENTO,VL_COMPRIMENTOINTERNO,VL_GRAMATURA,VL_LAP,FL_LAP_INTERNO,FL_LAP_NO_COMPR,VL_LARGPECA,VL_LARGURA,VL_LARGURAINTERNA,VL_MULTCOMP,VL_MULTLARG,VL_PESOCAIXA,VL_PESOPECA,VL_REFILOCOMPRIMENTO,VL_REFILOLARGURA,VL_REFUGOCLIENTE,VL_VINCOCOMP1,VL_VINCOCOMP2,VL_VINCOCOMP3,VL_VINCOCOMP4,VL_VINCOCOMP5,VL_VINCOLARG1,VL_VINCOLARG2,VL_VINCOLARG3,VL_CONSUMO_COR_TOTAL,FL_PROLONGLAP
133680,1,5094-1,1,518441,630166-9/518441,630166-9,0,1,1,1,1,3.000000,2,18.0,1,4200.0,A2S-B,1,150.0,150.0,1.346589,0.652167,0.673295,1.304334,0.566081,5.0,699.0,0.0,1413.0,400.0,416.0,0.0,0,0,933.0,953.0,300.0,2.0,1.0,240.0,0.0,15.0,20.0,13.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
134404,1,7716-1,1,797231,630384-9/797231,630384-9,0,1,1,1,1,3.000000,3,18.0,1,4800.0,A3-B,17,99.0,99.0,1.395385,0.453330,0.465128,1.359990,0.438370,6.0,621.0,0.0,1873.0,424.0,446.0,0.0,0,0,730.0,745.0,299.0,3.0,1.0,195.0,0.0,10.0,15.0,3.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,0.0
138847,1,5094-1,1,518441,632404-65/518441,632404-65,0,1,1,1,1,3.000000,2,18.0,1,4200.0,A2S-B,1,150.0,150.0,1.346589,0.652167,0.673295,1.304334,0.566081,5.0,699.0,0.0,1413.0,400.0,416.0,0.0,0,0,933.0,953.0,300.0,2.0,1.0,240.0,0.0,15.0,20.0,13.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
144160,1,7927,1,826030,634941-27/826030,634941-27,0,1,1,1,0,3.000000,4,50.0,4,8400.0,B2-B,18,80.0,80.0,1.491860,0.361920,0.372965,1.447680,0.358301,5.0,480.0,0.0,1940.0,583.0,414.0,0.0,0,0,754.0,769.0,315.0,4.0,1.0,148.0,0.0,20.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1140.0,0.0
144180,1,7927,1,826030,634941-45/826030,634941-45,0,1,1,1,0,3.000000,4,50.0,4,8400.0,B2-B,18,80.0,80.0,1.491860,0.361920,0.372965,1.447680,0.358301,5.0,480.0,0.0,1940.0,583.0,414.0,0.0,0,0,754.0,769.0,315.0,4.0,1.0,148.0,0.0,20.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1140.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406885,0,-1,0,130000,REP021984/130000,REP021984,0,0,0,1,0,0.714286,1,0.0,0,280.0,A1-DC,152,0.0,0.0,0.336000,0.336000,0.336000,0.336000,0.336000,0.0,1400.0,0.0,1400.0,1400.0,0.0,0.0,0,0,240.0,240.0,240.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,120.0,0.0,0.0,0.0
406886,0,-1,0,130000,REP021985/130000,REP021985,0,0,0,1,0,0.526316,1,0.0,0,190.0,A3-DC,152,0.0,0.0,0.336000,0.336000,0.336000,0.336000,0.336000,0.0,1400.0,0.0,1400.0,1400.0,0.0,0.0,0,0,240.0,240.0,240.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,120.0,0.0,0.0,0.0
406887,0,-1,0,130000,REP021986/130000,REP021986,0,0,0,1,0,0.927835,1,0.0,0,970.0,A1-DC,152,0.0,0.0,0.476000,0.476000,0.476000,0.476000,0.476000,0.0,1400.0,0.0,1400.0,1400.0,0.0,0.0,0,0,240.0,340.0,240.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,170.0,170.0,0.0,0.0,0.0
406888,0,-1,0,130000,REP021987/130000,REP021987,0,0,0,1,0,0.689655,1,0.0,0,290.0,A1-DC,152,0.0,0.0,0.392000,0.392000,0.392000,0.392000,0.392000,0.0,1400.0,0.0,1400.0,1400.0,0.0,0.0,0,0,240.0,280.0,240.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.0,140.0,0.0,0.0,0.0


In [23]:
import pandas as pd
import numpy as np

# Assumindo que df_pedidos_fil é seu dataframe processado

# ============================================================================
# 1. FEATURES AGREGADAS DE VINCOS
# ============================================================================

print('Criando features de vincos...')

# Colunas de vincos individuais
colunas_vinco_comp = ['VL_VINCOCOMP1', 'VL_VINCOCOMP2', 'VL_VINCOCOMP3', 'VL_VINCOCOMP4', 'VL_VINCOCOMP5']
colunas_vinco_larg = ['VL_VINCOLARG1', 'VL_VINCOLARG2', 'VL_VINCOLARG3']


# b) Soma total em milímetros
df_pedidos_fil['VL_VINCOS_TOTAL_MM'] = (
df_pedidos_fil[colunas_vinco_comp].sum(axis=1) +
df_pedidos_fil[colunas_vinco_larg].sum(axis=1)
)


Criando features de vincos...


In [24]:
df_pedidos_fil.head()

,CAT_ESPELHO,CD_FACA,CAT_FILME,CD_ITEM,CD_OP,CD_PEDIDO,FL_CONTROLE_ESPECIAL_IMPRESSAO,FL_AMARRADO,FL_TESTE_EXIGELAUDO,FL_REFILADO,FL_RESINAINTERNA,PERC_VAR_PEDIDA,QT_ARRANJO,VL_COBBINTMAXIMO,QT_NRCORES,QT_PEDIDA,CAT_COMPOSICAO,TX_TIPOABNT,VL_ALTURAINTERNA,VL_ALTURAINTERNA_REAL,VL_AREABRUTACHAPA,VL_AREABRUTAPECA,VL_AREABRUTAPECACOMREFILOS,VL_AREALIQUIDACHAPA,VL_AREALIQUIDAPECA,VL_COLUNAMINIMO,VL_COMPPECA,VL_COMPRESSAO,VL_COMPRIMENTO,VL_COMPRIMENTOINTERNO,VL_GRAMATURA,VL_LAP,FL_LAP_INTERNO,FL_LAP_NO_COMPR,VL_LARGPECA,VL_LARGURA,VL_LARGURAINTERNA,VL_MULTCOMP,VL_MULTLARG,VL_PESOCAIXA,VL_PESOPECA,VL_REFILOCOMPRIMENTO,VL_REFILOLARGURA,VL_REFUGOCLIENTE,VL_VINCOCOMP1,VL_VINCOCOMP2,VL_VINCOCOMP3,VL_VINCOCOMP4,VL_VINCOCOMP5,VL_VINCOLARG1,VL_VINCOLARG2,VL_VINCOLARG3,VL_CONSUMO_COR_TOTAL,FL_PROLONGLAP,VL_VINCOS_TOTAL_MM
133680,1,5094-1,1,518441,630166-9/518441,630166-9,0,1,1,1,1,3.0,2,18.0,1,4200.0,A2S-B,1,150.0,150.0,1.346589,0.652167,0.673295,1.304334,0.566081,5.0,699.0,0.0,1413.0,400.0,416.0,0.0,0,0,933.0,953.0,300.0,2.0,1.0,240.0,0.0,15.0,20.0,13.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
134404,1,7716-1,1,797231,630384-9/797231,630384-9,0,1,1,1,1,3.0,3,18.0,1,4800.0,A3-B,17,99.0,99.0,1.395385,0.453330,0.465128,1.359990,0.438370,6.0,621.0,0.0,1873.0,424.0,446.0,0.0,0,0,730.0,745.0,299.0,3.0,1.0,195.0,0.0,10.0,15.0,3.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,0.0,0.0
138847,1,5094-1,1,518441,632404-65/518441,632404-65,0,1,1,1,1,3.0,2,18.0,1,4200.0,A2S-B,1,150.0,150.0,1.346589,0.652167,0.673295,1.304334,0.566081,5.0,699.0,0.0,1413.0,400.0,416.0,0.0,0,0,933.0,953.0,300.0,2.0,1.0,240.0,0.0,15.0,20.0,13.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
144160,1,7927,1,826030,634941-27/826030,634941-27,0,1,1,1,0,3.0,4,50.0,4,8400.0,B2-B,18,80.0,80.0,1.491860,0.361920,0.372965,1.447680,0.358301,5.0,480.0,0.0,1940.0,583.0,414.0,0.0,0,0,754.0,769.0,315.0,4.0,1.0,148.0,0.0,20.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1140.0,0.0,0.0
144180,1,7927,1,826030,634941-45/826030,634941-45,0,1,1,1,0,3.0,4,50.0,4,8400.0,B2-B,18,80.0,80.0,1.491860,0.361920,0.372965,1.447680,0.358301,5.0,480.0,0.0,1940.0,583.0,414.0,0.0,0,0,754.0,769.0,315.0,4.0,1.0,148.0,0.0,20.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1140.0,0.0,0.0


In [26]:
# df_pedidos_fil["FL_TESTE_COBB_INT_MAX"] = df_pedidos_fil["FL_TESTE_COBB_INT_MAX"].apply(
#     lambda x: 1 if x > 0 else 0
# )

# df_pedidos_fil["FL_TESTE_COLUNA_MINIMO"] = df_pedidos_fil[
#     "FL_TESTE_COLUNA_MINIMO"
# ].apply(lambda x: 1 if x > 0 else 0)



# df_pedidos_fil["FL_TESTE_GRAMATURA"] = df_pedidos_fil[
#     "FL_TESTE_GRAMATURA"
# ].apply(lambda x: 1 if x > 0 else 0)


In [27]:
# a) Razão dimensional da CHAPA (antes da conversão)
if 'VL_COMPRIMENTO' in df_pedidos_fil.columns and 'VL_LARGURA' in df_pedidos_fil.columns:
    df_pedidos_fil['RAZAO_CHAPA_COMP_LARG'] = (
        df_pedidos_fil['VL_COMPRIMENTO'] / df_pedidos_fil['VL_LARGURA']
    ).replace([np.inf, -np.inf], np.nan)
    print(f'  ✅ RAZAO_CHAPA_COMP_LARG: média = {df_pedidos_fil["RAZAO_CHAPA_COMP_LARG"].mean():.2f}')

# b) Razão dimensional da PEÇA PLANIFICADA (pós-corte)
if 'VL_COMPPECA' in df_pedidos_fil.columns and 'VL_LARGPECA' in df_pedidos_fil.columns:
    df_pedidos_fil['RAZAO_PECA_COMP_LARG'] = (
        df_pedidos_fil['VL_COMPPECA'] / df_pedidos_fil['VL_LARGPECA']
    ).replace([np.inf, -np.inf], np.nan)
    print(f'  ✅ RAZAO_PECA_COMP_LARG: média = {df_pedidos_fil["RAZAO_PECA_COMP_LARG"].mean():.2f}')

# c) Volume interno da CAIXA MONTADA (em dm³ / litros)
colunas_volume = ['VL_COMPRIMENTOINTERNO', 'VL_LARGURAINTERNA', 'VL_ALTURAINTERNA']
if all(col in df_pedidos_fil.columns for col in colunas_volume):
    df_pedidos_fil['VOLUME_INTERNO'] = (
        df_pedidos_fil['VL_COMPRIMENTOINTERNO'] *
        df_pedidos_fil['VL_LARGURAINTERNA'] *
        df_pedidos_fil['VL_ALTURAINTERNA']
    ) / 1000000
    
    

  ✅ RAZAO_CHAPA_COMP_LARG: média = 2.54
  ✅ RAZAO_PECA_COMP_LARG: média = 1.42


In [28]:
colunas_remover = [
    # Identificadores
    "ID_CLIENTE",
    "FL_AMARRADO",
    # Consumo de cores (redundante com QT_NRCORES)
    "QT_CONSUMOCOR1",
    "QT_CONSUMOCOR2",
    "QT_CONSUMOCOR3",
    "QT_CONSUMOCOR4",
    # Duplicados
    "VL_ALTURAINTERNA_REAL",  # Duplicado de VL_ALTURAINTERNA
    # Áreas redundantes (manter só VL_AREALIQUIDAPECA)
    "VL_AREABRUTACHAPA",
    "VL_AREABRUTAPECA",
    "VL_AREABRUTAPECACOMREFILOS",
    "VL_AREALIQUIDACHAPA",
]

colunas_existentes_remover = [col for col in colunas_remover if col in df_pedidos_fil.columns]
df_pedidos_fil = df_pedidos_fil.drop(columns=colunas_existentes_remover)


In [29]:
df_pedidos_fil.to_parquet("tb_pedidos_process.parquet")